In [2]:
from Bio import Entrez

Entrez.email = "zeiberg.d@northeastern.edu"

import pandas as pd

from easydict import EasyDict

In [4]:
from tqdm.notebook import tqdm

import numpy as np
import pickle

In [5]:
def getSequenceFromNPID(npid):
    "Return the protein sequence from "
    handle = Entrez.efetch(db="protein",id=npid, rettype="fasta", retmode="text")
    lines = handle.readlines()
    lines = [l.strip() for l in lines]
    return "".join(lines[1:])

In [7]:
df = pd.read_csv("/ssdata/hgmd/HGMD_PRO_2019_1_hg19.vcf",delimiter="\t",header=14)

/home/dz/anaconda3/envs/bio/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
df

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,1,865595,CM1613956,A,G,.,.,CLASS=DM?;MUT=ALT;GENE=SAMD11;STRAND=+;DNA=NM_...
1,1,874491,CM1613954,C,T,.,.,CLASS=DM?;MUT=ALT;GENE=SAMD11;STRAND=+;DNA=NM_...
2,1,877523,CM1511864,C,G,.,.,CLASS=DM?;MUT=ALT;GENE=SAMD11;STRAND=+;DNA=NM_...
3,1,879286,CS1613955,A,C,.,.,CLASS=DM?;MUT=ALT;GENE=SAMD11;STRAND=+;DNA=NM_...
4,1,879375,CM1613953,C,T,.,.,CLASS=DM;MUT=ALT;GENE=SAMD11;STRAND=+;DNA=NM_1...
...,...,...,...,...,...,...,...,...
229156,Y,2655774,CD056857,CCCT,C,.,.,CLASS=DM;MUT=ALT;GENE=SRY;STRAND=-;DNA=NM_0031...
229157,Y,6931938,CM121018,G,C,.,.,CLASS=DM?;MUT=ALT;GENE=TBL1Y;STRAND=+;DNA=NM_0...
229158,Y,6938305,CM121019,C,T,.,.,CLASS=DM?;MUT=ALT;GENE=TBL1Y;STRAND=+;DNA=NM_0...
229159,Y,14847658,CD993525,TTAAG,T,.,.,CLASS=DM;MUT=ALT;GENE=USP9Y;STRAND=+;DNA=NM_00...


In [9]:
def getInfo(row):
    d = EasyDict()
    for vals in row["INFO"].split(";"):
        k,v = vals.split("=")
        d[k] = v
    pid, variant = d.PROT.split(":")
    pid,_ = pid.split(".")
    _,variant = variant.split(".")
    reference,location,missense = variant[0],int(variant[1:-1]),variant[-1]
    d.pid = pid
    d.reference = reference
    d.loc = location - 1
    d.missense = missense
    return d

In [10]:
 getInfo(df.loc[0])

{'CLASS': 'DM?',
 'MUT': 'ALT',
 'GENE': 'SAMD11',
 'STRAND': '+',
 'DNA': 'NM_152486.2:c.133A>G',
 'PROT': 'NP_689699.2:p.K45E',
 'DB': 'rs903331232',
 'PHEN': '"Retinitis_pigmentosa"',
 'RANKSCORE': '0.21',
 'pid': 'NP_689699',
 'reference': 'K',
 'loc': 44,
 'missense': 'E'}

In [11]:
def addSeqToInfo(info):
    info.seq = getSequenceFromNPID(info.pid)
    return info

In [12]:
protein_ids = set()
skipped = []
for i,r in tqdm(df.iterrows(),total=df.shape[0]):
    try:
        info = getInfo(r)
        protein_ids.add(info.pid)
    except:
        skipped.append(i)

  0%|          | 0/229161 [00:00<?, ?it/s]

In [14]:
sequences = EasyDict()

In [15]:
for npid in tqdm(protein_ids):
    sequences[npid] = getSequenceFromNPID(npid)

  0%|          | 0/9234 [00:00<?, ?it/s]

In [17]:
pickle.dump(sequences, open("/ssdata/hgmd/sequences.pkl","wb"))